In [2]:
import pandas
import databento

In [3]:
pandas.set_option('display.max_rows', 50)
pandas.set_option('display.min_rows', 20)

Load data from file

In [4]:
def get_path(ticker, filename=None):
    ticker = 'nvda'
    #filename = 'xnas-itch-20240105.mbo.dbn.zst'
    return f'/databento/data/mbo/{ticker}/XNAS-20240622-WPPRESG4BH/{filename}'

In [5]:
full_path = get_path('nvda', filename='xnas-itch-20240105.mbo.dbn.zst')
full_path

'/databento/data/mbo/nvda/XNAS-20240622-WPPRESG4BH/xnas-itch-20240105.mbo.dbn.zst'

In [6]:
stored_data = databento.DBNStore.from_file(full_path)
df: pandas.DataFrame = stored_data.to_df().reset_index()

In [7]:
df

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
0,2024-01-05 09:00:00.034832298+00:00,2024-01-05 09:00:00.034665350+00:00,160,2,11667,A,B,225.11,41,0,2803,130,166948,274050,NVDA
1,2024-01-05 09:00:00.207384375+00:00,2024-01-05 09:00:00.207209487+00:00,160,2,11667,A,B,478.98,100,0,8955,130,174888,275072,NVDA
2,2024-01-05 09:00:00.214426993+00:00,2024-01-05 09:00:00.214260186+00:00,160,2,11667,A,A,480.78,100,0,12639,130,166807,275517,NVDA
3,2024-01-05 09:00:00.217115284+00:00,2024-01-05 09:00:00.216948664+00:00,160,2,11667,C,A,480.78,100,0,12639,130,166620,275795,NVDA
4,2024-01-05 09:00:00.220495504+00:00,2024-01-05 09:00:00.220328153+00:00,160,2,11667,A,A,483.30,100,0,15775,130,167351,276222,NVDA
5,2024-01-05 09:00:00.220911026+00:00,2024-01-05 09:00:00.220744075+00:00,160,2,11667,C,A,483.30,100,0,15775,130,166951,276263,NVDA
6,2024-01-05 09:00:00.226255123+00:00,2024-01-05 09:00:00.226087928+00:00,160,2,11667,A,A,483.84,100,0,17947,130,167195,276759,NVDA
7,2024-01-05 09:00:00.291889915+00:00,2024-01-05 09:00:00.291723309+00:00,160,2,11667,C,A,483.84,100,0,17947,0,166606,278910,NVDA
8,2024-01-05 09:00:00.291889915+00:00,2024-01-05 09:00:00.291723309+00:00,160,2,11667,A,A,483.57,100,0,29299,130,166606,278910,NVDA
9,2024-01-05 09:00:00.292379010+00:00,2024-01-05 09:00:00.292213221+00:00,160,2,11667,C,A,483.57,100,0,29299,130,165789,278975,NVDA


In [8]:
for column in df.columns:
    print(column)

ts_recv
ts_event
rtype
publisher_id
instrument_id
action
side
price
size
channel_id
order_id
flags
ts_in_delta
sequence
symbol


`df_aux` is a copy of `df`, the original data loaded from file.

Documentation on the `action` column:

- Trades (`T`) are followed by a Fill (`F`) and then Cancel (`C`)
- Trades do not have an associated order id, however if a Trade message is followed by a Fill and a Cancel which both have the same order id then this value can be used as an order id for the original Trade

**Create new columns in `df_aux`:**

- `trade_action_fill` - for rows which are Trades, this is the value of the `action` column for the Fill row which follows
- `trade_action_cancel` - for rows which are Trades, this is the value of the `action` column for the Cancel row which follows
- `trade_order_id_fill` - for rows which are trades, this is the value of the `order_id` column for the Fill row which follows
- `trade_order_id_cancel` - for rows which are Trades, this is the value of the `order_id` column for the Cancel row which follows

In [10]:
df_aux = df.copy()
df_aux['trade_action_fill'] = df_aux['action'].shift(-1)
df_aux['trade_action_cancel'] = df_aux['action'].shift(-2)
df_aux['trade_order_id_fill'] = df_aux['order_id'].shift(-1)
df_aux['trade_order_id_cancel'] = df_aux['order_id'].shift(-2)
df_aux[['action', 'trade_action_fill', 'trade_action_cancel', 'side', 'price', 'size', 'order_id', 'trade_order_id_fill', 'trade_order_id_cancel']]

,action,trade_action_fill,trade_action_cancel,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel
0,A,A,A,B,225.11,41,2803,8955.0,12639.0
1,A,A,C,B,478.98,100,8955,12639.0,12639.0
2,A,C,A,A,480.78,100,12639,12639.0,15775.0
3,C,A,C,A,480.78,100,12639,15775.0,15775.0
4,A,C,A,A,483.30,100,15775,15775.0,17947.0
5,C,A,C,A,483.30,100,15775,17947.0,17947.0
6,A,C,A,A,483.84,100,17947,17947.0,29299.0
7,C,A,C,A,483.84,100,17947,29299.0,29299.0
8,A,C,A,A,483.57,100,29299,29299.0,29595.0
9,C,A,C,A,483.57,100,29299,29595.0,29595.0


I want to know: How many Trade rows are followed by a Fill then a Cancel vs how many are not?

In [11]:
# How many rows are Trades, which are followed by a Fill and then a Cancel
# The Fill and Cancel rows do not necessarily have to be related
df_aux_trades_with_expected_schema = (
    df_aux[
        (df_aux['action'] == 'T') &
        (df_aux['trade_action_fill'] == 'F') &
        (df_aux['trade_action_cancel'] == 'C')
    ]
)
print(f'Number of trades followed by Fill and Cancel')
print(len(df_aux_trades_with_expected_schema))

Number of trades followed by Fill and Cancel


68205

In [12]:
# How many rows in total are Trades?
df_aux_trades = (
    df_aux[
        (df_aux['action'] == 'T')
    ]
)
print(f'Number of Trades')
print(len(df_aux_trades))

Number of Trades


103898

In [13]:
# How many rows in total are Trades which are not followed by a Fill and then a Cancel
print(f'Number of Trades not followed by Fill and Cancel')
print(len(df_aux_trades) - len(df_aux_trades_with_expected_schema))

Number of Trades not followed by Fill and Cancel


35693

In [22]:
# How many rows are Trades which are not followed by a Fill and then a Cancel.
# (Same as the above, but with an explicit method of selecting these rows.)
# This dataframe is known as the "bad schema" dataframe.
# The number should match the above number.
df_aux_trades_with_bad_schema = (
    df_aux[
        (df_aux['action'] == 'T') &
        (
            (df_aux['trade_action_fill'] != 'F') |
            (df_aux['trade_action_cancel'] != 'C')
        )
    ]
)
print(f'Number of Trades not followed by Fill and Cancel (explicit calculation)')
print(len(df_aux_trades_with_bad_schema))

Number of Trades not followed by Fill and Cancel (explicit calculation)
35693


# Check for related rows in `df_aux_trades_with_expected_schema`

`df_aux_trades_with_expected_schema` contains all rows which are Trades followed by a Fill and then a Cancel. But the Fills and Cancels may not be related to each other, or the original trade. Need to perform explicit check for this.

Perform vectorized checks for the data in other columns.

This is a list of columns in the original dataframe:

```
ts_recv, ts_event, rtype, publisher_id, instrument_id, action, side, price, size, channel_id, order_id, flags, ts_in_delta, sequence, symbol
```

Options:
- Create a dataframe from `df` which contains all Trades which are followed by Fills and Cancels, as well as those fills and cancels (select by schema first), then check that the rows contain the expected data
- Create a dataframe from `df` which contains all events, with augmented columns indicating if the row is a Trade followed by a Fill and then a Cancel where the Trade, Fill and Cancel data matches (at least as far as possible using the relevant columns). (The order id field of a trade will *not* match because Trades do not have an associated order id.)

In [15]:
print(', '.join(df.columns))

ts_recv, ts_event, rtype, publisher_id, instrument_id, action, side, price, size, channel_id, order_id, flags, ts_in_delta, sequence, symbol


Need to move this somewhere else - just demonstrating that `ts_in_delta` doesn't appear to be the same as `ts_recv` - `ts_event`

In [27]:
df2 = df.copy()
df2['ts_in_delta_2'] = (df2['ts_recv'] - df2['ts_event']).dt.nanoseconds
df2['ts_in_delta_diff'] = df2['ts_in_delta'] - df2['ts_in_delta_2']
df2.loc[df2['ts_in_delta_diff'] != 0][['ts_in_delta', 'ts_in_delta_2', 'ts_in_delta_diff']][0:5]

,ts_in_delta,ts_in_delta_2,ts_in_delta_diff
0,166948,948,166000
1,174888,888,174000
2,166807,807,166000
3,166620,620,166000
4,167351,351,167000


In [66]:
columns = list(filter(lambda column: column not in ['action', 'flags', 'ts_in_delta'], df))
print(', '.join(columns))

ts_recv, ts_event, rtype, publisher_id, instrument_id, side, price, size, channel_id, order_id, sequence, symbol


Back to the previous workflow

In [45]:
# Create dataframe with augmented columns
df_augmented = df.copy()
df_aug = df_augmented

df_aug_columns = list(df_aug.columns)
df_aug_columns_relevant = list(filter(lambda column: column not in ['action', 'flags', 'ts_in_delta'], df_aug_columns))

for column in df_aug_columns:
    new_column_1 = f'{column}_shift_1'
    df_aug[new_column_1] = df_aug[column].shift(-1, fill_value=None)
    new_column_2 = f'{column}_shift_2'
    df_aug[new_column_2] = df_aug[column].shift(-2, fill_value=None)

    if column not in df_aug_columns_relevant:
        continue

    new_column_1_match = f'{new_column_1}_match'
    df_aug[new_column_1_match] = df_aug[column] == df_aug[new_column_1]

    new_column_2_match = f'{new_column_2}_match'
    df_aug[new_column_2_match] = df_aug[column] == df_aug[new_column_2]

#print('\n'.join(df_aug.columns))

Now find the set of rows which are Trades followed by Fill and then Cancel where:
- All other columns match
- All other columns do not match

In [58]:
df_aug_TFC = df_aug.copy()
df_aug_TFC['TFC_select'] = (df_aug_TFC['action'] == 'T') & (df_aug_TFC['action_shift_1'] == 'F') & (df_aug_TFC['action_shift_2'] == 'C')
df_aug_TFC['TFC_select_shift_d1'] = df_aug_TFC['TFC_select'].shift(1)
df_aug_TFC['TFC_select_shift_d2'] = df_aug_TFC['TFC_select'].shift(2)
df_aug_TFC['TFC_select_any'] = (
   (df_aug_TFC['TFC_select']) |
   (df_aug_TFC['TFC_select_shift_d1']) |
   (df_aug_TFC['TFC_select_shift_d2'])
)
df_aug_TFC_selected = df_aug_TFC[df_aug_TFC['TFC_select_any'] == True].copy()

In [59]:
df_aug_TFC_selected[['action', 'action_shift_1', 'action_shift_2']]

,action,action_shift_1,action_shift_2
343,T,F,C
344,F,C,A
345,C,A,A
363,T,F,C
364,F,C,C
365,C,C,C
410,T,F,C
411,F,C,C
412,C,C,T
414,T,F,C


In [60]:
print(df_aug_TFC_selected.columns)

Index(['ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id',
       'action', 'side', 'price', 'size', 'channel_id', 'order_id', 'flags',
       'ts_in_delta', 'sequence', 'symbol', 'ts_recv_shift_1',
       'ts_recv_shift_2', 'ts_recv_shift_1_match', 'ts_recv_shift_2_match',
       'ts_event_shift_1', 'ts_event_shift_2', 'ts_event_shift_1_match',
       'ts_event_shift_2_match', 'rtype_shift_1', 'rtype_shift_2',
       'rtype_shift_1_match', 'rtype_shift_2_match', 'publisher_id_shift_1',
       'publisher_id_shift_2', 'publisher_id_shift_1_match',
       'publisher_id_shift_2_match', 'instrument_id_shift_1',
       'instrument_id_shift_2', 'instrument_id_shift_1_match',
       'instrument_id_shift_2_match', 'action_shift_1', 'action_shift_2',
       'side_shift_1', 'side_shift_2', 'side_shift_1_match',
       'side_shift_2_match', 'price_shift_1', 'price_shift_2',
       'price_shift_1_match', 'price_shift_2_match', 'size_shift_1',
       'size_shift_2', 'size_shift_1_match

In [64]:
# Figure out which rows have the expected matches
# Do not match on the action, these are different because rows follow T, F, C sequence
# Do not match on price, the C price may be different to the T and F price, although T and F should match
# Do not match on side, T appears to be on the opposite side
# Do not match on order id, as Trades have `order_id` = 0
# Do not match on any of the `match_all*` column names, these are output columns not input to this operation
df_aug_TFC_selected['match_all'] = True

for column in df_aug_TFC_selected.columns:
    if not 'match' in column:
        print(f'skip: column={column}')
        continue

    if 'action' in column:
        print(f'skip: column={column}')
        continue

    # If this is disabled, then the output file will contain some rows. These
    # rows will be for Trades which were filled at a different price from the
    # corresponding Cancel price.
    # if 'price' in column:
    #     print(f'skip: column={column}')
    #     continue

    # TODO: implement matching on reverse side
    if 'side' in column:
        print(f'skip: column={column}')
        continue

    if 'order_id' in column:
        print(f'skip: column={column}')
        continue

    if 'match_all' in column:
        print(f'skip: column={column}')
        continue

    print(f'apply match: column={column}')
    df_aug_TFC_selected['match_all'] = df_aug_TFC_selected['match_all'] & df_aug_TFC_selected[column]

# The `match_all` value is only valid for Trade rows
# Shift `match_all` to create values for Fill and Cancel rows
df_aug_TFC_selected['match_all_shift_d1'] = df_aug_TFC_selected['match_all'].shift(1)
df_aug_TFC_selected['match_all_shift_d2'] = df_aug_TFC_selected['match_all'].shift(2)

# Keep rows which have `match_all` == True for the corresponding Trade row
df_aug_TFC_selected['match_all_any'] = (
    (df_aug_TFC_selected['match_all'] & (df_aug_TFC_selected['action'] == 'T')) |
    (df_aug_TFC_selected['match_all_shift_d1'] & (df_aug_TFC_selected['action'] == 'F')) |
    (df_aug_TFC_selected['match_all_shift_d2'] & (df_aug_TFC_selected['action'] == 'C'))
)

skip: column=ts_recv
skip: column=ts_event
skip: column=rtype
skip: column=publisher_id
skip: column=instrument_id
skip: column=action
skip: column=side
skip: column=price
skip: column=size
skip: column=channel_id
skip: column=order_id
skip: column=flags
skip: column=ts_in_delta
skip: column=sequence
skip: column=symbol
skip: column=ts_recv_shift_1
skip: column=ts_recv_shift_2
apply match: column=ts_recv_shift_1_match
apply match: column=ts_recv_shift_2_match
skip: column=ts_event_shift_1
skip: column=ts_event_shift_2
apply match: column=ts_event_shift_1_match
apply match: column=ts_event_shift_2_match
skip: column=rtype_shift_1
skip: column=rtype_shift_2
apply match: column=rtype_shift_1_match
apply match: column=rtype_shift_2_match
skip: column=publisher_id_shift_1
skip: column=publisher_id_shift_2
apply match: column=publisher_id_shift_1_match
apply match: column=publisher_id_shift_2_match
skip: column=instrument_id_shift_1
skip: column=instrument_id_shift_2
apply match: column=inst

In [65]:
# Save any rows which did not match to file
df_aug_TFC_selected[df_aug_TFC_selected['match_all_any'] != True][0:20].to_csv('df_aug_TFC_selected_where_match_all_is_False.csv', index=False)

#### `df_aux_trades_with_expected_schema` contains all the Trades followed by a Fill then a Cancel

This dataframe can be saved to a file, and it will contain all the relevant trade data

Except: The rows may not be related. Before performing this selection, should check to see if rows are related!

In [19]:
df_aux_trades_with_expected_schema[
    [   'instrument_id', 'side', 'price', 'size',
        'action', 'trade_action_fill', 'trade_action_cancel', 'order_id', 'trade_order_id_fill', 'trade_order_id_cancel']
]

,instrument_id,side,price,size,action,trade_action_fill,trade_action_cancel,order_id,trade_order_id_fill,trade_order_id_cancel
343,11667,B,480.43,1,T,F,C,0,174323.0,174323.0
363,11667,A,480.30,70,T,F,C,0,167971.0,167971.0
410,11667,A,480.35,56,T,F,C,0,186087.0,186087.0
414,11667,A,480.30,130,T,F,C,0,167971.0,167971.0
472,11667,A,480.36,36,T,F,C,0,188779.0,188779.0
743,11667,B,480.00,25,T,F,C,0,260071.0,260071.0
779,11667,B,480.16,4,T,F,C,0,252271.0,252271.0
820,11667,B,480.30,20,T,F,C,0,194395.0,194395.0
875,11667,B,480.30,1,T,F,C,0,292899.0,292899.0
894,11667,A,480.37,200,T,F,C,0,293299.0,293299.0


In [23]:
df_aux_trades_with_bad_schema[
    [   'instrument_id', 'side', 'price', 'size',
        'action', 'trade_action_fill', 'trade_action_cancel', 'order_id', 'trade_order_id_fill', 'trade_order_id_cancel']
]

,instrument_id,side,price,size,action,trade_action_fill,trade_action_cancel,order_id,trade_order_id_fill,trade_order_id_cancel
225,11667,N,480.36,20,T,T,T,0,0.0,0.0
226,11667,N,480.35,20,T,T,C,0,0.0,171147.0
227,11667,N,480.35,10,T,C,A,0,171147.0,171471.0
359,11667,N,480.36,20,T,T,T,0,0.0,0.0
360,11667,N,480.35,40,T,T,T,0,0.0,0.0
361,11667,N,480.31,20,T,T,T,0,0.0,0.0
362,11667,N,480.31,50,T,T,F,0,0.0,167971.0
398,11667,N,480.36,20,T,A,A,0,186003.0,186087.0
608,11667,N,480.29,5,T,A,C,0,215159.0,188855.0
892,11667,N,480.39,20,T,T,T,0,0.0,0.0


In [21]:
df_aux_trades[
    [   'instrument_id', 'side', 'price', 'size',
        'action', 'trade_action_fill', 'trade_action_cancel', 'order_id', 'trade_order_id_fill', 'trade_order_id_cancel']
]

,instrument_id,side,price,size,action,trade_action_fill,trade_action_cancel,order_id,trade_order_id_fill,trade_order_id_cancel
225,11667,N,480.36,20,T,T,T,0,0.0,0.0
226,11667,N,480.35,20,T,T,C,0,0.0,171147.0
227,11667,N,480.35,10,T,C,A,0,171147.0,171471.0
343,11667,B,480.43,1,T,F,C,0,174323.0,174323.0
359,11667,N,480.36,20,T,T,T,0,0.0,0.0
360,11667,N,480.35,40,T,T,T,0,0.0,0.0
361,11667,N,480.31,20,T,T,T,0,0.0,0.0
362,11667,N,480.31,50,T,T,F,0,0.0,167971.0
363,11667,A,480.30,70,T,F,C,0,167971.0,167971.0
398,11667,N,480.36,20,T,A,A,0,186003.0,186087.0


This is now redundant

back to original code...

In [26]:
df_aux_trades_with_expected_schema = df_aux_trades_with_expected_schema.copy()

In [27]:
# df_aux_trade_action_match_fill = df_aux['trade_action_fill'] == 'F'
# df_aux['trade_action_match_fill'] = df_aux_trade_action_match_fill
# #df_aux_trade_action_match_fill

df_aux_trades_with_expected_schema['trade_action_match_fill'] = df_aux_trades_with_expected_schema['trade_action_fill'] == 'F'

In [28]:
# df_aux_trade_action_match_cancel = df_aux['trade_action_cancel'] == 'C'
# df_aux['trade_action_match_cancel'] = df_aux_trade_action_match_cancel
# #df_aux_trade_action_match_cancel

df_aux_trades_with_expected_schema['trade_action_match_cancel'] = df_aux_trades_with_expected_schema['trade_action_cancel'] == 'C'

In [29]:
# df_aux_trade_action_match_both = df_aux_trade_action_match_fill & df_aux_trade_action_match_cancel
# df_aux['trade_action_match_both'] = df_aux_trade_action_match_both
# #df_aux_trade_action_match_both

df_aux_trades_with_expected_schema['trade_action_match_both'] = (
    df_aux_trades_with_expected_schema['trade_action_match_fill'] &
    df_aux_trades_with_expected_schema['trade_action_match_cancel']
)

In [30]:
# df_aux_trade_action_match_both.value_counts()

df_aux_trades_with_expected_schema['trade_action_match_both'].value_counts()

trade_action_match_both
True    68205
Name: count, dtype: int64

In [33]:
columns_to_select = [
    "action",
    "trade_action_fill",
    "trade_action_cancel",
    "trade_action_match_fill",
    "trade_action_match_cancel",
    "trade_action_match_both",
    "side",
    "price",
    "size",
    "order_id",
    "trade_order_id_fill",
    "trade_order_id_cancel",
]

In [34]:
# df_aux[columns_to_select]

df_aux_trades_with_expected_schema[columns_to_select]

,action,trade_action_fill,trade_action_cancel,trade_action_match_fill,trade_action_match_cancel,trade_action_match_both,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel
343,T,F,C,True,True,True,B,480.43,1,0,174323.0,174323.0
363,T,F,C,True,True,True,A,480.30,70,0,167971.0,167971.0
410,T,F,C,True,True,True,A,480.35,56,0,186087.0,186087.0
414,T,F,C,True,True,True,A,480.30,130,0,167971.0,167971.0
472,T,F,C,True,True,True,A,480.36,36,0,188779.0,188779.0
743,T,F,C,True,True,True,B,480.00,25,0,260071.0,260071.0
779,T,F,C,True,True,True,B,480.16,4,0,252271.0,252271.0
820,T,F,C,True,True,True,B,480.30,20,0,194395.0,194395.0
875,T,F,C,True,True,True,B,480.30,1,0,292899.0,292899.0
894,T,F,C,True,True,True,A,480.37,200,0,293299.0,293299.0


In [35]:
#df_aux_matched_rows = df_aux[df_aux['trade_action_match_both'] == True][columns_to_select]
df_aux_matched_rows = df_aux_trades_with_expected_schema[df_aux_trades_with_expected_schema['trade_action_match_both'] == True][columns_to_select]
df_aux_matched_rows[df_aux_matched_rows['trade_order_id_fill'] != df_aux_matched_rows['trade_order_id_cancel']] # important that this is empty

,action,trade_action_fill,trade_action_cancel,trade_action_match_fill,trade_action_match_cancel,trade_action_match_both,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel


In [36]:
more_columns_to_select = columns_to_select + ['order_id_replace']
more_columns_to_select

['action',
 'trade_action_fill',
 'trade_action_cancel',
 'trade_action_match_fill',
 'trade_action_match_cancel',
 'trade_action_match_both',
 'side',
 'price',
 'size',
 'order_id',
 'trade_order_id_fill',
 'trade_order_id_cancel',
 'order_id_replace']

Create the column `order_id_replace`. It takes the value of `trade_order_id_fill` which is the `order_id` value from the corresponding "fill" row. It is used to associate an order id with a trade event, which otherwise does not have one.

In [39]:
df_aux_trades_with_expected_schema['order_id_replace'] = 0
df_aux_trades_with_expected_schema.loc[
    df_aux_trades_with_expected_schema['trade_action_match_both'] == True, 'order_id_replace'
] = (
    df_aux_trades_with_expected_schema.loc[
        df_aux_trades_with_expected_schema['trade_action_match_both'] == True, 'trade_order_id_fill'
    ].astype(int)
)
#df_aux.dtypes

In [41]:
df_aux_trades_with_expected_schema[more_columns_to_select]

,action,trade_action_fill,trade_action_cancel,trade_action_match_fill,trade_action_match_cancel,trade_action_match_both,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel,order_id_replace
343,T,F,C,True,True,True,B,480.43,1,0,174323.0,174323.0,174323
363,T,F,C,True,True,True,A,480.30,70,0,167971.0,167971.0,167971
410,T,F,C,True,True,True,A,480.35,56,0,186087.0,186087.0,186087
414,T,F,C,True,True,True,A,480.30,130,0,167971.0,167971.0,167971
472,T,F,C,True,True,True,A,480.36,36,0,188779.0,188779.0,188779
743,T,F,C,True,True,True,B,480.00,25,0,260071.0,260071.0,260071
779,T,F,C,True,True,True,B,480.16,4,0,252271.0,252271.0,252271
820,T,F,C,True,True,True,B,480.30,20,0,194395.0,194395.0,194395
875,T,F,C,True,True,True,B,480.30,1,0,292899.0,292899.0,292899
894,T,F,C,True,True,True,A,480.37,200,0,293299.0,293299.0,293299


In [42]:
df_aux_trades_with_expected_schema.to_csv('df_aux_trades_with_expected_schema.csv')

Below code no longer required to be run

In [ ]:
df_aux_trades = df_aux[df_aux['action'] == 'T']
df_aux_trades[more_columns_to_select].iloc[0:1000].to_csv('df_aux.csv')

In [84]:
df_aux['order_id'] = df_aux['order_id_replace']

In [ ]:
df_aux

In [46]:
original_columns = df.columns
original_columns

Index(['ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id',
       'action', 'side', 'price', 'size', 'channel_id', 'order_id', 'flags',
       'ts_in_delta', 'sequence', 'symbol'],
      dtype='object')

Run this code to save the trades dataframe to csv

In [47]:
# df_trades = df_aux_trades[original_columns]
# df_trades.to_csv('xnas-itch-20240105.mbo.dbn.trades.csv', index=False)

df_aux_trades_with_expected_schema[original_columns].to_csv('xnas-itch-20240105.mbo.dbn.trades.csv', index=False)

# Older Code (still important - extracts the Orders)

In [ ]:
df_aux[df_aux['trade_action_fill'] != 'F']

In [15]:
# don't run
df_trades = df_aux[df_aux['action'] == 'T']
df_trades.to_csv('xnas-itch-20240105.mbo.dbn.trades.csv', index=False)

In [48]:
from mbo_data_package import filter_df_remove_fill_cancel_pairs

df_orders = filter_df_remove_fill_cancel_pairs(df)

In [49]:
df_orders

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
0,2024-01-05 09:00:00.034832298+00:00,2024-01-05 09:00:00.034665350+00:00,160,2,11667,A,B,225.11,41,0,2803,130,166948,274050,NVDA
1,2024-01-05 09:00:00.207384375+00:00,2024-01-05 09:00:00.207209487+00:00,160,2,11667,A,B,478.98,100,0,8955,130,174888,275072,NVDA
2,2024-01-05 09:00:00.214426993+00:00,2024-01-05 09:00:00.214260186+00:00,160,2,11667,A,A,480.78,100,0,12639,130,166807,275517,NVDA
3,2024-01-05 09:00:00.217115284+00:00,2024-01-05 09:00:00.216948664+00:00,160,2,11667,C,A,480.78,100,0,12639,130,166620,275795,NVDA
4,2024-01-05 09:00:00.220495504+00:00,2024-01-05 09:00:00.220328153+00:00,160,2,11667,A,A,483.30,100,0,15775,130,167351,276222,NVDA
5,2024-01-05 09:00:00.220911026+00:00,2024-01-05 09:00:00.220744075+00:00,160,2,11667,C,A,483.30,100,0,15775,130,166951,276263,NVDA
6,2024-01-05 09:00:00.226255123+00:00,2024-01-05 09:00:00.226087928+00:00,160,2,11667,A,A,483.84,100,0,17947,130,167195,276759,NVDA
7,2024-01-05 09:00:00.291889915+00:00,2024-01-05 09:00:00.291723309+00:00,160,2,11667,C,A,483.84,100,0,17947,0,166606,278910,NVDA
8,2024-01-05 09:00:00.291889915+00:00,2024-01-05 09:00:00.291723309+00:00,160,2,11667,A,A,483.57,100,0,29299,130,166606,278910,NVDA
9,2024-01-05 09:00:00.292379010+00:00,2024-01-05 09:00:00.292213221+00:00,160,2,11667,C,A,483.57,100,0,29299,130,165789,278975,NVDA


In [23]:
df_orders.to_csv('xnas-itch-20240105.mbo.dbn.orders.csv', index=False)

In [25]:
df_orders.iloc[208176-10:208237+10]

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
219128,2024-01-05 14:37:33.797028077+00:00,2024-01-05 14:37:33.796861795+00:00,160,2,11667,A,A,486.33,100,0,48824947,130,166282,44454334,NVDA
219129,2024-01-05 14:37:33.843151345+00:00,2024-01-05 14:37:33.842984915+00:00,160,2,11667,C,B,485.48,25,0,48796979,130,166430,44455569,NVDA
219130,2024-01-05 14:37:33.843153772+00:00,2024-01-05 14:37:33.842987637+00:00,160,2,11667,A,B,485.59,25,0,48826127,130,166135,44455570,NVDA
219131,2024-01-05 14:37:33.866281659+00:00,2024-01-05 14:37:33.866115203+00:00,160,2,11667,A,A,485.95,3,0,48826579,130,166456,44455917,NVDA
219132,2024-01-05 14:37:33.900320009+00:00,2024-01-05 14:37:33.900153604+00:00,160,2,11667,A,A,485.96,25,0,48828083,130,166405,44456813,NVDA
219133,2024-01-05 14:37:33.939642307+00:00,2024-01-05 14:37:33.939470463+00:00,160,2,11667,C,B,485.59,25,0,48826127,0,171844,44459106,NVDA
219134,2024-01-05 14:37:33.939642307+00:00,2024-01-05 14:37:33.939471046+00:00,160,2,11667,A,A,485.94,25,0,48831495,128,171261,44459107,NVDA
219135,2024-01-05 14:37:34.084778199+00:00,2024-01-05 14:37:34.084612230+00:00,160,2,11667,A,B,472.00,20,0,48838395,130,165969,44465026,NVDA
219136,2024-01-05 14:37:34.123896147+00:00,2024-01-05 14:37:34.123730513+00:00,160,2,11667,C,B,485.57,25,0,48815935,130,165634,44465550,NVDA
219137,2024-01-05 14:37:34.133602923+00:00,2024-01-05 14:37:34.133436328+00:00,160,2,11667,C,B,450.96,50,0,9121543,130,166595,44465869,NVDA


In [24]:
!ls -alh

total 521M
drwxr-xr-x  2 ecb ecb 4.0K Jun 25 13:49 .
drwxr-xr-x 15 ecb ecb 4.0K Jun 24 17:31 ..
-rw-r--r--  1 ecb ecb 275K Jun 25 10:21 explore.ipynb
-rw-r--r--  1 ecb ecb  17K Jun 25 13:50 extract_order_book_input.ipynb
-rw-r--r--  1 ecb ecb 123K Jun 24 18:09 order_id_838222951.csv
-rwxr-xr-x  1 ecb ecb 1.3K Jun 24 17:28 read_mbo_data.py
-rw-r--r--  1 ecb ecb 508M Jun 25 13:50 xnas-itch-20240105.mbo.dbn.orders.csv
-rw-r--r--  1 ecb ecb  13M Jun 25 13:25 xnas-itch-20240105.mbo.dbn.trades.csv
